# Lambda notebook metalanguage documentation
### Author: Kyle Rawlins

__Last updated__: 12/24/14

The core of the lambda notebook is a flexible and extensible typed logical metalanguage implemented in python -- the __Lambda Notebook Metalanguage (LNM)__.  The metalanguage is python-like, and can be interacted with either by cell magics, or by a python API.  Expressions of the metalanguage are parsed into python representations that can be interacted with programmatically.  There are three main aspects of the metalanguage to document: (i) the type system, (ii) the class hierarchy that supports this implementation, and (iii) the parser for writing metalanguage expressions directly.

At the technical level, the core unit of the implementation of LNM is the TypedExpr class.  Every expression is an instance of this class or one of its subclasses.  A TypedExpr has a type, and consists of an 'operator' slot, and 0 or more argument slots.  The operator slot is, in most cases, filled by a logical operator or a function.  In a few special cases (terms), it is used to store a variable or constant name.  All `TypedExpr` objects implement routines to be printed in an ipython notebook.  Also, `repr` returns a string that is guaranteed to be a parseable LNM expression that will lead to the same object.

# Interacting with the metalanguage

The metalanguage has two units: statements and expressions.  Expressions evaluate to metalanguage objects (of class _TypedExpr_ or a subclass), and are the main unit of the metalanguage.  For example, the following lines are each expressions.  The first one is a variable of type _e_, the second applies a variable to a constant property, and the third applies a constant of type _e_ to a lambda expression.

    x_e
    P_<e,t>(x_e)
    (L x_e : Cat(x))(Josie_e)

__Statements__ (currently) are only equality statements, and do not have a value -- they update the lexicon, variable namespace, and python namespace with an expression.  The thing to the right of `=` is a LNM expression.  Two things can be assigned values: variables names, and lexical items.  A lexical item is notated by surround the expression with `||` on each side.  Each of the following is a statement that assigns one of the above examples to a variable:

    x = x_e
    tmp = P_<e,t>(x_e)
    c = (L x_e : Cat(x))(Josie_e)
    
The following example illustrates a lexical item assignment:

    ||cat|| = L x_e : Cat(x)

There are two ways to process metalanguage: via ipython magics, and via python functions.  To process a statement, there is an ipython 'magic' named __lamb__.  This magic works in both cell and inline mode.  In cell mode, the first line of an ipython cell should be `%%lamb`, and any following lines will be treated as metalanguage statements.  The following cell interprets the above 4 examples of LNM statements.

In [3]:
%%lamb
x = x_e
tmp = P_<e,t>(x_e)
c = (L x_e : Cat(x))(Josie_e)
||cat|| = L x_e : Cat(x)

INFO (meta): Coerced guessed type t for 'Cat_t' into <e,t>, to match argument 'x_e'
INFO (meta): Coerced guessed type t for 'Cat_t' into <e,t>, to match argument 'x_e'


${x}_{e}\:=\:{x}_{e}$<br />
${tmp}_{t}\:=\:{P}({x}_{e})$<br />
${c}_{t}\:=\:[\lambda{} x_{e} \: . \: {Cat}({x}_{e})]({Josie}_{e})$<br />
$[\![\mathbf{\text{cat}}]\!]^{}_{\langle{}e,t\rangle{}} \:=\: \lambda{} x_{e} \: . \: {Cat}({x}_{e})$

Each statement interpreted by the __lamb__ magic has four effects: (i) the expression to the right of the equals sign is interpreted by the LNM parser and converted to a python object of class `TypedExpr`; (ii) the resulting object is printed; (iii) if the expression is a lexical item, an item with that name is added to the lexicon, and (iv) a variable with the name to the left of `=` is exported to the python namespace if possible.

  * _Technical note_: if the variable name is shadowed by something in the namespace, exporting (currently) fails.  (TODO: how to access?)
  
In inline mode, a single line of a cell is prefaced with `%lamb`; the line is interpreted as a statement in the same way described above, with one caveat: the result is only printed if the inline magic is the last line in the cell.

In [4]:
print("This is normal python code here")
print(repr(tmp)) # this accesses the variable defined above via python code
%lamb ||dog|| = L x_e : Dog(x)
print("This is still python code")

INFO (meta): Coerced guessed type t for 'Dog_t' into <e,t>, to match argument 'x_e'


This is normal python code here
P_<e,t>(x_e)
This is still python code


In [5]:
dog # let's print this just to make sure.  As a reminder, this cell is ordinary python code.

⟦dog⟧ = (λ x_e: Dog_<e,t>(x_e))

__Expressions__ can be input using statements as above, evaluated using the `%te` line magic, or directly evaluated in python using the function `te` (automatically imported from `lamb.lang`).  

The `te` line magic evaluates a single expression.  This has the main effect, if used as the last line of a cell, of just printing the output.

  * As a side effect, this magic stores the result of each evaluation to the python variable `_llast`.

This can be mixed with python to a very limited extent, in order to do variable assignment.  Note that this command completely bypasses the lexicon, no matter how it is used.

In [6]:
%te L x_e : Dog(x)

INFO (meta): Coerced guessed type t for 'Dog_t' into <e,t>, to match argument 'x_e'


(λ x_e: Dog_<e,t>(x_e))

In [7]:
_llast

(λ x_e: Dog_<e,t>(x_e))

In [8]:
d = %te L x_e : Dog(x)
d

INFO (meta): Coerced guessed type t for 'Dog_t' into <e,t>, to match argument 'x_e'


(λ x_e: Dog_<e,t>(x_e))

The `te` python function behaves similarly: it parses an LNM expression provided as a string, and returns an appropriate `TypedExpr`.  The result can be manipulated just like any python object.

In [9]:
d = te("L x_e : Dog(x)") # directly assign a python variable this LNM expression.  This bypasses the lexicon.
d

INFO (meta): Coerced guessed type t for 'Dog_t' into <e,t>, to match argument 'x_e'


(λ x_e: Dog_<e,t>(x_e))

In [10]:
d(te("Joe_e"))

((λ x_e: Dog_<e,t>(x_e)))(Joe_e)

In [11]:
d(te("Joe_e")).reduce_all() # this performs beta reduction on any lambda expressions

Dog_<e,t>(Joe_e)

# Types

The type system will be familiar to anyone who has used a typed lambda calculus or logical system such as Ty2, and this documentation assumes some basic familiarity with typed lambda calculi.  The standard definition of this sort of type hierarchy looks something like this, where rule 2 defines functional types:

 1. $e$ and $t$ are types.  (_atomic types_)
 2. If $\alpha$ and $\beta$ are types, so is $\langle\alpha,\beta\rangle$. (_non-atomic types_)
 3. Nothing else is a type.

The type system here is extensible and designed for practical use, so it isn't useful to consider such a strict definition, but the basic ideas hold.  A type system in the lambda notebook consists of a set of atomic types and non-atomic types.  The library provides three atomic types: 

 * $t$ (truth-values)
 * $e$ (entities)
 * $n$ (numbers)

Adding new atomic types is easy; see e.g. the neo-Davidsonian notebook.  The library also provides several non-atomic types:

 * $\langle\alpha,\beta\rangle$: functional types, where the left is the input type and the right the output type.
 * $(\alpha_0,...\alpha_n)$: tuple types, for building tuples from atomic (or non-atomic) types.  In practice, what are often treated in logic as n-ary constants are treated here as functions from n-ary tuples to truth-values.
 * $\{\alpha\}$: set types, for sets of elements of type $\alpha$.
 
The library also provides a mechanism for limited type uncertainty and inference in the form of undetermined types, notated with '?'.  (_TODO_: more detail)

All types are interpreted and parsed relative to a type system, and in `meta` a current type system is defined; this can be accessed with `meta.get_type_system()`.  A convenience function `meta.tp` (imported into the notebook's namespace) calls the type system's parser.  By default, the type system is `types.under_system`.

In [12]:
t = tp("<<e,t>,t>")
t

<<e,t>,t>

In [13]:
t.functional()

True

In [14]:
t.left

<e,t>

In [15]:
t.right

t

In [16]:
t.right.functional()

False

In [17]:
tp("<(e,e),t>")

<(e,e),t>

(_TODO_: document unification, equality testing)

# Metalanguage

There are four key kinds of TypedExprs to worry about: terms (variables and constants), lambda expressions, operators, and binding operators.  (Actually lambda expressions are a special case of the latter, but are important enough to separate out.)

In [18]:
meta.suppress_constant_predicate_type = False # normally, predicate types are hidden for prettier output.  But it is helpful here to be able to see them

### Terms

A term consists simply of a name and optionally a type.  Capitalized names are constants, and lower-case names are variables.  If no type is specified, it will guess.

In [19]:
%te P(x)

INFO (meta): Coerced guessed type t for 'P_t' into <e,t>, to match argument 'x_e'


P_<e,t>(x_e)

The code below demonstrates an example expression consisting of a function and argument; the function is the constant `P` and the argument the variable `x`.  Note that no types are specified; everything in the metalanguage has a type so various shortcut inference processes come into play.  Variables default to type $e$ and constants to type $t$, but it guesses because of the argument that $P$ should be a higher type.

In [20]:
e = %te P(x)
ltx_print(e, e.type) # function for printing multiple ipython latex display objects

INFO (meta): Coerced guessed type t for 'P_t' into <e,t>, to match argument 'x_e'


${P}_{\langle{}e,t\rangle{}}({x}_{e})$<br />$t$<br />

Types can be explicitly specified by putting a `_` after a term.

In [21]:
%te P_<e,t>(x_e)

P_<e,t>(x_e)

In [22]:
e == te("P_<e,t>(x_e)") # is e equivalent to the explicitly typed form?

True

The objects generated by parsing have an API, which is not comprehensively documented here.  The following cells briefly demonstrate the part of the API for accessing sub-expressions.  Every complex TypedExpr is structured into an operator (`op`) and 0 or more arguments (`args`).

In [23]:
print("The operator's python type is: " + str(e.op.__class__))
ltx_print("The operator's LNM representation and type is: ", e.op, e.op.type)

The operator's python type is: <class 'lamb.meta.TypedTerm'>


The operator's LNM representation and type is: <br />${P}_{\langle{}e,t\rangle{}}$<br />$\langle{}e,t\rangle{}$<br />

In [24]:
print("The (only) argument's python type is: " + str(e.args[0].__class__))
ltx_print("The argument's LNM representation and type is: ", e.args[0], e.args[0].type)

The (only) argument's python type is: <class 'lamb.meta.TypedTerm'>


The argument's LNM representation and type is: <br />${x}_{e}$<br />$e$<br />

In [25]:
# the .op and .args notation can be simplified just to indices, where [0] is the op.
ltx_print(e[0], e[1])

${P}_{\langle{}e,t\rangle{}}$<br />${x}_{e}$<br />

When commas are used in the argument to a function, the argument is treated as an n-ary tuple.  This is the standard way to construct n-ary 'predicates' in LNM.

In [26]:
%te P(x,y)

INFO (meta): Coerced guessed type t for 'P_t' into <(e,e),t>, to match argument '(x_e, y_e)'


P_<(e,e),t>(x_e, y_e)

There is an API for constructing expressions in python, without using LNM syntax.  The following cells provide some examples (but not comprehensive documentation).

To construct terms programmatically, use the term factory function instead of instantiating classes directly, which can be shortened to `meta.term`.  It is also often convenient to use the type parser here, though the factory function is fairly flexible.

In [27]:
e3 = meta.term("P", tp("<(e,e),t>"))
e3

P_<(e,e),t>

In [28]:
meta.term("P_<(e,e),t>")

P_<(e,e),t>

Many python operators are overloaded to properly handle TypedExprs, most importantly function calling.

In [29]:
meta.term("P_<(e,e),t>")(meta.term("x_e"), meta.term("y_e"))

P_<(e,e),t>(x_e, y_e)

In [30]:
meta.term("P_<e,<e,t>>")(meta.term("x_e"))(meta.term("y_e"))

P_<e,<e,t>>(x_e)(y_e)

### Lambda expressions

Lambda expressions (class: `LFun`) consist of a typed variable (i.e. lower-case TypedTerm) and a body, which is a TypedExpr.  Instances of that variable name in the body (and not under the scope of some lower binder) are interpreted as bound by that lambda operator, and the types must much (or a TypeMismatch exception will result).  The type of the whole is the type of a function from the variable's type to the body's type.  They are notated as `L var_type : body`.  The parser will except several alternatives for `L`, including the more pythonic `lambda`.

In [31]:
e3 = %te L y_e : P(y)
ltx_print(e3, e3.type)

INFO (meta): Coerced guessed type t for 'P_t' into <e,t>, to match argument 'y_e'


$\lambda{} y_{e} \: . \: {P}_{\langle{}e,t\rangle{}}({y}_{e})$<br />$\langle{}e,t\rangle{}$<br />

Function-argument expressions can be easily written in the expected way using parenthesis:  (parenthesis around the function are obligatory)

In [32]:
%te (L y_e : P(y))(x_e)

INFO (meta): Coerced guessed type t for 'P_t' into <e,t>, to match argument 'y_e'


((λ y_e: P_<e,t>(y_e)))(x_e)

Note that beta-reduction does not happen automatically (although type-checking does).  There are a number of ways to force beta-reduction.  The simplest for displaying a reduction is to add `reduce` preceded and followed by a space to a `%te` magic.  The others are via python calls, and are explained below.

  * technical note: if you are using a variable named `reduce`, just ensure there is no space following it.

In [33]:
%te reduce (L y_e : P(y))(x_e)

INFO (meta): Coerced guessed type t for 'P_t' into <e,t>, to match argument 'y_e'


P_<e,t>(x_e)

As with terms, the python function call notation works correctly.  'Correct' in this case, though, means that it will construct a composite TypedExpr with the lambda expression as the operator, and the argument as the operand.

In [34]:
e3(te("x_e"))

((λ y_e: P_<e,t>(y_e)))(x_e)

To actually apply the argument to the function (and force beta reduction), there are two ways.  You can construct the expression using `apply`, or call the `reduce_all()` function on any `TypedExpr`.

In [35]:
e3.apply(te("x_e"))

P_<e,t>(x_e)

In [36]:
e4 = e3(te("x_e")).reduce_all()
e4

P_<e,t>(x_e)

Quick sanity check: this should be equal to the first example `e` defined in the term section.

In [37]:
e4 == e

True

There is one important difference between the original `e` and `e4`; some aspects of the derivational history are saved.

In [38]:
e4.derivation

In [39]:
e.derivation

A somewhat more complicated example:

In [40]:
e5 = %te L y_e : L x_e : P(x,y)
e5

INFO (meta): Coerced guessed type t for 'P_t' into <(e,e),t>, to match argument '(x_e, y_e)'


(λ y_e: (λ x_e: P_<(e,e),t>(x_e, y_e)))

In [41]:
e6 = e5(te("A_e"))(te("B_e"))
e6

((λ y_e: (λ x_e: P_<(e,e),t>(x_e, y_e))))(A_e)(B_e)

In [42]:
e6.reduce_all()

P_<(e,e),t>(B_e, A_e)

Note that (as you'd expect), variable names of bound variables are not guaranteed to stay the same on lambda application.  Variable names of free variables are safe.  In the following example, `x` is free in the main argument in the definition of `e7`.  On application, the bound `x` must be renamed to avoid collision with the free `x`.

In [43]:
%te reduce (L p_t : L x_e : p & P(x))(P(x))

INFO (meta): Coerced guessed type t for 'P_t' into <e,t>, to match argument 'x_e'
INFO (meta): Coerced guessed type t for 'P_t' into <e,t>, to match argument 'x_e'


(λ x1_e: (P_<e,t>(x_e) & P_<e,t>(x1_e)))

In [44]:
# another example of a collision where both variables are bound.  The one that is locally bound gets renamed.
%te reduce L x_e : (L p_t : L x_e : p & P(x))(P(x))

INFO (meta): Coerced guessed type t for 'P_t' into <e,t>, to match argument 'x_e'
INFO (meta): Coerced guessed type t for 'P_t' into <e,t>, to match argument 'x_e'


(λ x_e: (λ x1_e: (P_<e,t>(x_e) & P_<e,t>(x1_e))))

### Operator expressions

Logical operators (and numeric operators) are supported using python syntax.  This should be mostly straightforward for anyone familiar with classical logic.  As you might expect, types are enforced.  The parsed syntax is (with a few exceptions) the same as the python syntax.

Representations are always bracketed unambiguously, and python operator precedence defines bracketing.  This is mostly straightforward with a few caveats:

 * __equality__: the operators `==` and `!=` will not behave as expected in the metalanguage.  Instead use '`<=>`' and '`=/=`' in LNM expressions.
 * __logical operators__: similarly, python's logical operators (`&&` etc) do not work on LNM.  Use the python bitwise operator symbols instead.
 * __precedence__: the operators currently all use python precedence, even in LNM, which can lead to counterintuitive results.  Parenthesis should be used to disambiguate, rather than relying on precedence.

The material implication operator uses `>>`.  In the metalanguage (but not python), `==>` will also work.

In [45]:
%te p_t & (q_t | r_t)

(p_t & (q_t | r_t))

In [46]:
%te p_t >> ~q_t

(p_t >> ~q_t)

Many operators are overloaded to work in python as well; the following instances of `&`, `|`, `>>`, and `~` are python.

In [47]:
te("p_t") & (te("q_t") | te("r_t"))

(p_t & (q_t | r_t))

In [48]:
te("~p_t") >> ~te("q_t")

(~p_t >> ~q_t)

### Binding expressions

A number of operators use a similar syntax to lambda expressions, though with different type structures.  Existential and universal quantification are expressions of type $t$, that bind a variable of some type and whose body is type $t$.  Sets can also be defined using a similar notation, but their overall type is the type of a set whose elements are the variable's type.

It is relatively easy to add operators like this; see the demo notebook on the definite article/iota operator for how.

In [49]:
%te Exists x_e : P(x)

INFO (meta): Coerced guessed type t for 'P_t' into <e,t>, to match argument 'x_e'


(Exists x_e: P_<e,t>(x_e))

In [50]:
%te Forall x_e : P(x)

INFO (meta): Coerced guessed type t for 'P_t' into <e,t>, to match argument 'x_e'


(Forall x_e: P_<e,t>(x_e))

In [51]:
%te Set f_<e,t> : Exists x_e : f(x)

(Set f_<e,t>: (Exists x_e: f_<e,t>(x_e)))

# Appendix: metalanguage class hierarchy

TypedExpr: base class for all typed expressions.

 * BinaryOpExpr: wrapper classes for logical operators.
    - BinaryAndExpr (python / LNM: '`&`')
    - BinaryOrExpr (python / LNM: '`|`')
    - BinaryArrowExpr (python / LNM: '`>>`'.  __Warning__: this takes very high precedence, parens are recommended.  LNM only: '`==>`')
    - BinaryBiarrowExpr (LNM: '`<=>`' at type t)
    - BinaryNeqExpr (python/LNM: '`^`' or '`%`'  LNM only: '`=/=`')
    - BinaryGenericEqExpr (LNM: '`<=>`' at types other than t. __Warning__: '`==`' does not work.)
    - SetContains (python/LNM: '`<<`'.  __Warning__: precedence is tricky, always use parens.)
    - numeric expressions: BinaryLExpr, BinaryLeqExpr, BinaryGeqExpr, BinaryGExpr, BinaryPlusExpr, BinaryMinusExpr, Binary DivExpr, BinaryTimesExpr, BinaryExpExpr
 * UnaryOpExpr: Unary operator expressions.
    - UnaryNegExpr: negation. (Python/LNM: '`~`')
    - UnaryNegativeExpr: numeric unary negative operator.  (Python/LNM: '`-`')
 * BindingOp: class for operators that bind a variable (notated `v` below) in their scope.  For many of these, there are shorthand expressions not documented here, and the unicode will also work.
    - ConditionSet: set defined in terms of a condition on some variable.  (LNM: '`Set v : `'
    - ForallUnary: $\forall$ operator. (LNM: '`Forall v :`')
    - ExistsUnary: $\exists$ operator. (LNM: '`Exists v :`')
    - IotaUnary: $\iota$ operator (e.g. for definite descriptions).  (LNM: '`Iota v :`')
    - LFun: $\lambda$ expression, describing a function. (LNM: '`L v :`')
 * ListedSet: set defined by listing elements.
 * Tuple: a typed tuple.  (Elements can be of arbitrary type, and needn't be uniform.)
 * TypedTerm: a typed term (variable or constant).
    - CustomTerm: class for dealing with some special ways of printing terms, common in the linguistics literature.
 